In [91]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
CALCULATED_CARD_DRAW_UPS = dataiku.Dataset("CALCULATED_CARD_DRAW_UPS")
CALCULATED_CARD_DRAW_UPS_df = CALCULATED_CARD_DRAW_UPS.get_dataframe()

CALCULATED_CARD_DRAW_DOWNS = dataiku.Dataset("CALCULATED_CARD_DRAW_DOWNS")
CALCULATED_CARD_DRAW_DOWNS_df = CALCULATED_CARD_DRAW_DOWNS.get_dataframe()

In [92]:
df_down = CALCULATED_CARD_DRAW_DOWNS_df
df_up = CALCULATED_CARD_DRAW_UPS_df

df_down.sort_values(['CUSTOMER'], inplace=True)
df_up.sort_values(['CUSTOMER'], inplace=True)

print(len(df_down), "draw downs")
print(len(df_up), "draw ups")

34557 draw downs
48169 draw ups


In [93]:
df_down.head()

,CUSTOMER,DRAW_DOWN_DATE,MEAN_DD,STD_DD,ACTIVE_CARD_MAX
0,1 AFFORDABLE LAWN MAINTENANCE,2022-02-01 00:00:00+00:00,2.000000,0.000000,3.0
9999,1 BRUDA 3 SISTAS LLC,2021-09-01 00:00:00+00:00,NaN,NaN,1.0
9998,1 G R V LLC,2019-08-01 00:00:00+00:00,0.500000,0.577350,7.0
9997,1 KC LOGISTICS,2021-10-01 00:00:00+00:00,NaN,NaN,1.0
9996,1 PRIORITY ENVIRONMENTAL SERVICES LLC,2021-01-01 00:00:00+00:00,6.333333,0.492366,7.0


In [125]:
import string

class Draw_Down_Customer:

    def __init__(self, name, draw_down_date, mean_dd, std_dd, active_card_max):

        self._common_words = ['PIZZA', 'MANAGEMENT', 'THERAPEUTICS', 'USA', 'INC', 'US', 'EQUIPMENT', 'MEDICAL', 'SYSTEMS',
                             'ANIMAL', 'HEALTH', 'LLC', 'CORPORATION', 'BRANDS', 'TIRE', 'RUBBER', 'COUNTRY', 'CORP',
                              'PHARMACY','INC', 'RESTAURANTS', 'CONTAINER', 'AMERICA', 'APPLICATIONS', 'TECHNOLOGY',
                              'INSURANCE', 'FARM','CREDIT', 'SERVICES', 'SERVICE', 'ACCOUNT', 'GENERAL', 'PARTS',
                              'INTL', 'FLAVORS', 'HOLDINGS', 'FOOD','INDUSTRIES', 'LP', 'FLEET', 'MEDICAL', 'PHARMA',
                             'GLOBAL', 'PIPELINE', 'WHEELS', 'BIOSCIENCES', 'SSI', 'SPRINGS', 'NORTH', 'MARINE', 'HOLDING',
                              'TECHNOLOGIES','GROUP', 'PHARMACEUTICAL', 'NA', 'USA', 'COMPANY', 'RAIL', 'PARTNERS', 'BROS',
                              'CO', 'PHARMACEUTICALS', 'ENERGY', 'DISTRIBUTION', 'DENTAL', 'SPECIALTIES', 'OPERATIONS',
                              'COMPANY', 'THE', 'MOUNTAIN', 'TRANS', 'FUEL', 'AMERICAN', 'HOMES', 'GAS',
                             'AFFORDABLE', 'LAWN', 'MAINTENANCE','LOGISTICS','INC','LLC','CO','CORP','LTD','LIMITED',
                              'IN','COMPANY','COMPANIES','VENTURES','TECHNOLOGIES','WORKS','AND','THE','OF','COMPANY',
                              'SYSTEMS','PROPERTY','EXPRESS','SONS','BROTHERS','BUILDING','SERVIC','SERVICE','SERVICES',
                              'CONSTRUCTION','CONSTRUCTIONS','ELECTRIC','ELECTIC','ELECTRICAL','ELECTRONICS','GROUPE',
                              'GROUP','SOLUTIONS','PLUMBING','ENTERPRISES','TRANSPORT','TRANSPORTATION','SYSTEMS',
                              'MANAGEMENT','CONTRACTING','ASSOCIATES','CONSULTING','CONTRAC','CONTRACTORS','CONSTRUCTORS',
                              'SECURITY','INDUSTRIES','EXPRESS','SONS','PROPERTIES','INVESTMENTS','INVESTMENT',
                              'CORPORATION','BUILDERS','ENTERPRISE','STORE','INDUSTRIAL','AUTOMOTIVE','ENGINEERING',
                              'INTERNATIONAL','MEDICAL','MOTORS','STATE','COMMUNICATIONS','COMMUNICATION','DELIVERY',
                              'COMMERCIAL','REFRIGERATION','BUSINESS','HOUSING','DEPARTMENT','TECHNOLOGY','FOODS',
                              'PRODUCTIONS','MANUFACTURING','CONTRACTOR','DISTRIBUTORS','SYSTEM','ENTERTAINMENT',
                              'HOSPITAL','OPERATIONS','EXTERIORS','ASSOCIATED','FOUNDATIONS','LABORATORIES','BLACK',
                              'DECK','ENERGY','BUS','CONCRETE','CONTROL','CONTROLS','FIRE','WHEELS','TEST','KM','WINDOW',
                              'CLEANING','STEEMER','DEZURE','WHEELS','TEST','EQUIPMENT','COURIER','SANITARY',
                              'PAINT','PARTS','ELECTRICA','PRIORITY', 'ENVIRONMENTAL','WAY', 'AUTO', 'FREIGHT',
                              'WITH', 'NATURE',  'AMBULANCE', 'FINANCIAL', 'SALES', 'HOUR', 'CENTERS','MEMORY', 'LANE', 
                              'CENTRAL','STREET', 'SUPPLY', 'CENTER', 'TRANSPORTING', 'JUNK', '1ST', 'ACTION', 'HEATING', 
                              'AIR','ADVANCED', 'ALARM', 'INFUSION',  'BAPTIST', 'CHURCH', 'CALL', 'TOW', 'RECOVERY', 
                              'CHOICE', 'COLLATERAL', 'LANDSCAPE', 'CLASS','ALARM', 'CORPORATE', 'TULSA']

        self.CUSTOMER = name
        self.DRAW_DOWN_DATE = draw_down_date
        self.ACTIVE_CARD_MAX = active_card_max

        self.MATCHING_CUSTOMERS = []
        self.PERCENT_DIFFERENCE = []
        self.DAYS_DIFFERENCE = []
        self.DRAW_UP_DATE = []

        # remove punctuation
        c_str = name.translate(str.maketrans('', '', string.punctuation))

        f = c_str.split()
        self.WORD_LIST = []
        for w in f:
            if w not in self._common_words:
                self.WORD_LIST.append(w)

    def Match_Draw_Up_Customer(self, name, draw_up_date, mean_du, std_du, active_card_max):

        if (self.CUSTOMER == name):
            # exact match, already captured
            return

        c_str = name.translate(str.maketrans('', '', string.punctuation))

        f = c_str.split()

        check_list = []
        for w in f:
            if (w not in self._common_words) and (len(w)>1) and (not w.isnumeric()):
                check_list.append(w)

        percent_diff = round((abs(self.ACTIVE_CARD_MAX - active_card_max) / ((self.ACTIVE_CARD_MAX+active_card_max)/2)),2)

        #date_format = "%Y-%m-%d"
        #d1_date = datetime.strptime(draw_up_date.astype(str), date_format)
        #d2_date = datetime.strptime(self.DRAW_DOWN_DATE.astype(str), date_format)

        delta_between_drop_and_rise = round(abs((draw_up_date-self.DRAW_DOWN_DATE).days)/30.,0)

        for w_to_check in check_list:
            for w in self.WORD_LIST:
                if w_to_check==w:

                    if not name in(self.MATCHING_CUSTOMERS) and(delta_between_drop_and_rise<=4)and(percent_diff<=0.5) :
                        self.MATCHING_CUSTOMERS.append(name)
                        self.PERCENT_DIFFERENCE.append(percent_diff)
                        self.DAYS_DIFFERENCE.append(delta_between_drop_and_rise)
                        self.DRAW_UP_DATE.append(draw_up_date)
                        break;

In [0]:
idx = 0
_customers = []
verbose = True

max_idx = 1000

for index, row in df_down.iterrows():

    idx+=1

    customer = row['CUSTOMER']
    draw_down_date = row['DRAW_DOWN_DATE']
    mean_dd = row['MEAN_DD']
    std_dd = row['STD_DD']
    active_card_max = row['ACTIVE_CARD_MAX']

    c = Draw_Down_Customer(customer, draw_down_date, mean_dd, std_dd, active_card_max)

    _customers.append(c)

    if max_idx>0:
        if idx>max_idx:
            break;

idx = 0

_direct_customer = []
_direct_match = []
_direct_draw_up_date = []

_multiple_customer = []
_multiple_matches = []
_multiple_drop_dates = []

_no_match_customer = []

for c in _customers:

    for index_up, row_up in df_up.iterrows():

        idx+=1

        customer = row_up['CUSTOMER']
        draw_up_date = row_up['DRAW_UP_DATE']
        mean_du = row_up['MEAN_DU']
        std_du = row_up['STD_DU']
        active_card_max = row_up['ACTIVE_CARD_MAX']

        c.Match_Draw_Up_Customer(customer, draw_up_date, mean_du, std_du, active_card_max)

    if len(c.MATCHING_CUSTOMERS)==1:

        _direct_customer.append(c.CUSTOMER)
        _direct_match.append(c.MATCHING_CUSTOMERS[0])
        _direct_draw_up_date.append(c.DRAW_UP_DATE[0])

        if verbose:
            print()
            print("DIRECT")
            print(c.CUSTOMER, c.WORD_LIST)
            print(c.MATCHING_CUSTOMERS)
            print(c.PERCENT_DIFFERENCE)
            print(c.DAYS_DIFFERENCE)
            print("=====")
            print()

    elif len(c.MATCHING_CUSTOMERS)>1:

        _multiple_customer.append(c.CUSTOMER)
        _multiple_matches.append(c.MATCHING_CUSTOMERS)
        _multiple_drop_dates.append(c.DRAW_UP_DATE)

        if verbose:
            print()
            print("MULTIPLE")
            print(c.CUSTOMER, c.WORD_LIST)
            print(c.MATCHING_CUSTOMERS)
            print(c.PERCENT_DIFFERENCE)
            print(c.DAYS_DIFFERENCE)
            print("=====")
            print()
    else:
        _no_match_customer.append(c.CUSTOMER)

print(idx)
print()

print(len(_direct_customer), "direct matches")
print(len(_multiple_matches), "multiple matches")
print(len(_no_match_customer), "no match customers")


DIRECT
11TH HOUR BUSINESS CENTERS ['11TH']
['11TH HOUR BUSINESS SOLUTIONS LLC']
[0.5]
[1.0]
=====


DIRECT
1844 JUNK RAT ['1844', 'RAT']
['JUNK RAT LLC']
[0.0]
[1.0]
=====


MULTIPLE
1ST JUDICIAL DISTRICT ATTORNEY ['JUDICIAL', 'DISTRICT', 'ATTORNEY']
['ANSLEY SCHOOL DISTRICT 44', 'BAYLOR COUNTY HOSPITAL DISTRICT INC', 'BLOOMSBURG AREA SCHOOL DISTRICT', 'CABAZON WATER DISTRICT', 'CAMERON PARISH WATERWORKS DISTRICT 10', 'CASCADE SCHOOL DISTRICT', 'CHARLES CITY COMMUNITY SCHOOL DISTRICT', 'CHESTER AREA SCHOOL DISTRICT', 'COLORADO CITY UNIFIED SCHOOL DISTRICT NO14', 'EDCOUCHELSA INDEPENDENT SCHOOL DISTRICT PUBLIC FACILITY CORPORATION', 'EDGEWOOD SOIL  WATER DISTRICT', 'FAIRVIEW SCHOOL DISTRICT 84', 'HABITAT FOR HUMANITY CAPITAL DISTRICT INC', 'HAMILTON LAKE CONSERVANCY DISTRICT', 'HARDIN COUNTY EMERGENCY SERVICES DISTRICT 2', 'INDIAN HILL EXEMPTED VILLAGE SCHOOL DISTRICT', 'JAL SCHOOL DISTRICT', 'JAMESTOWN PUBLIC SCHOOL DISTRICT 1', 'JANE LEW PUBLIC SERVICE DISTRICT', 'JEFFERSON COUNTY SO


MULTIPLE
2 G FARMS LLC ['2', 'G', 'FARMS']
['ALDERWOOD FARMS INC', 'AMR FARMS LLC', 'BELL FARMS TRUCKING INC', 'BUTTREY FARMS', 'CAROLINA FRESH FARMS LLC', 'CHRISTOPHER BLEDSO FARMS LLC', 'CR FARMS', 'D  D FARMS LLC', 'D  D KAMLA FARMS', 'D R FARMS', 'DAVID DAVIS FARMS INC', 'DORAZIO FARMS INC', 'DREAM CATCHER FARMS', 'DRP FARMS', 'HONEY URBAN FARMS LLC', 'J  F FARMS LANDSCAPING INC', 'JOE DAN FARMS', 'JUNO VIEW FARMS INC', 'KRZEWINA FARMS LLC', 'LANGER FARMS LLC', 'LYNN CREEK FARMS LLC', 'MCDONOGH FARMS INC', 'NEIL BASSETTI FARMS LLC', 'RECORD BUCK FARMS INC', 'SWACKHAMMER FARMS LTD', 'THEUNISSEN FARMS PARTNERSHIP', 'WEAVER FARMS']
[0.4, 0.4, 0.0, 0.0, 0.4, 0.4, 0.4, 0.4, 0.0, 0.0, 0.4, 0.0, 0.0, 0.0, 0.4, 0.0, 0.0, 0.0, 0.4, 0.0, 0.4, 0.4, 0.4, 0.4, 0.0, 0.4, 0.0]
[1.0, 4.0, 4.0, 3.0, 1.0, 1.0, 3.0, 1.0, 1.0, 3.0, 4.0, 2.0, 3.0, 2.0, 3.0, 1.0, 1.0, 3.0, 3.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 4.0]
=====


MULTIPLE
2 H M INCORPORATED ['2', 'H', 'M', 'INCORPORATED']
['RNA OF ANN ARBOR


MULTIPLE
21ST CENTURY ALARM DATA ['21ST', 'CENTURY', 'DATA']
['21ST CENTURY ALARMDATA INC', 'SUMMIT ELECTRIC  DATA INC']
[0.0, 0.33]
[1.0, 4.0]
=====


MULTIPLE
21ST CENTURY FLOORS ['21ST', 'CENTURY', 'FLOORS']
['BANTER FLOORS  MORE INC', 'BREWER CUSTOM FLOORS', 'CENTURY EXCAVATION', 'CENTURY LEASING CORPORATION', 'CROFTBECK FLOORS INC', 'EPOXY FLOORS BY WELCH LLC', 'INCREDIBLE FLOORS AND CONSTRUCTION SERVICES INC', 'SPECIALTY FLOORS INC']
[0.25, 0.15, 0.44, 0.15, 0.13, 0.15, 0.0, 0.44]
[2.0, 1.0, 4.0, 1.0, 1.0, 0.0, 2.0, 2.0]
=====


MULTIPLE
21ST CENTURY POOL & PATIO ['21ST', 'CENTURY', 'POOL', 'PATIO']
['21ST CENTURY POWER SOLUTIONS LLC', 'ADMIRAL POOL SERVICE INC', 'ATLANTIS POOL PLASTERING INC', 'AUSTIN POOL PLASTERING LLC', 'BEES POOL SERVICE INC', 'CENTURY LLC', 'CENTURY MAINTENANCE  SUPPLY CORP', 'COASTAL CUSTOM POOL LLC', 'COLONIAL POOL AND SPA', 'DENSCOT SWIMMING POOL AND LANDSCAPE SERVICES LLC', 'DESIGN POOL  SPA LTD', 'EAST BAY POOL SERVICE INC', 'FREEDOM POOL SERVICES', '


MULTIPLE
3 LITTLE BUILDERS ['3', 'LITTLE']
['FROM HOUSE TO HOME OF LITTLE ROCK LLC', 'LITTLE ONES  COMPANY LLC', 'LITTLE SISTERS OF THE POOR INC', 'S  KS LITTLE CHICKS DAYCARE INC', 'THE QUINTIN LITTLE COMPANY INC']
[0.0, 0.4, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 1.0, 1.0]
=====


MULTIPLE
3 MACK TRUCKING LLC ['3', 'MACK', 'TRUCKING']
['ALEXEYCALA TRUCKING', 'BEST DUMP TRUCKING INC.', 'BURKE RANCH TRUCKING INC', 'BUSHMAN TRUCKING LLC', 'CAPITAL TRUCKING LLC', 'DAN STORJOHANN TRUCKING', "ERIC'S TRUCKING EAS LLC", 'EUROTECH CONSTRUCTION  TRUCKING INC', 'GASS TRUCKING', 'GLOBAL EXPRESS TRUCKING CORPORATION', 'HOLMES TRUCKING  EXCAVATING LLC', 'INTENSE TRUCKING LLC', 'J F PEPPERS TRUCKING LLC', 'K & R DAY TRUCKING INC', 'LAGEWORKS TRUCKING AND EXCAVATING LLC', 'M  M TRUCKING OF ASHLEY COUNTY LLC', 'MIDAMERICAN TRUCKING INC INC', 'MILLCREEK TRUCKING LLC', 'OLD ROUTE TRUCKING LLC', 'P & H TRUCKING & DISPOSAL I(04', 'ROBERT TRUCKING LLC', 'SATURDAYS TRUCKING LLC', 'SILVER BULLET TRUCKING LLC', 'SO


MULTIPLE
3TP UNIQUE NATRAL HAIR ['3TP', 'UNIQUE', 'NATRAL', 'HAIR']
["BRIT'S HAIR BAR", 'ELEGANTLY UNIQUE BUNDLES LLC', 'HAIR ART NETWORK', 'HAIR FORCE 1', 'J GLAM HAIR EXTENSIONS LLC', 'LACE LUXURY HAIR LLC', 'LONDON HAIR KOLLECTION LLC', 'LOVE MICA HAIR COLLECTION LLC', 'LUXE HAIR CENTRAL LLC', 'LUXURY LIFESTYLE HAIR LLC', 'MANI SHAW HAIR CO', 'NATURAL ALLURE SKIN  HAIR LLC', 'THE DIFFERENCE HAIR SALON', 'TURN N HEADS HAIR STUDIO LLC', 'UNIQUE AIR LLC', 'UNIQUE DESIGN  PROPERTY LLC', 'UNIQUE TAXES LLC', 'UNIQUE TREATS LLC', 'UNIQUE WELDING AND CONTRACTING LLC', 'VICKYS HAIR CLOSET LLC', 'VIRGIN RAWMANE HAIR LLC', 'VIXENS HAIR BAR LLC', 'XTENDED HAIR CO LLC']
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 3.0, 4.0, 0.0, 0.0, 2.0, 4.0, 0.0, 1.0, 3.0, 0.0, 0.0, 1.0, 4.0, 3.0, 1.0, 1.0, 4.0, 1.0, 4.0, 4.0, 1.0, 4.0]
=====


MULTIPLE
3W SPECIALTY SERVICES LLC ['3W', 'SPECIALTY']
['ARIZONA SPECIALTY DEMOLITION LLC'

...


MULTIPLE
A GANG MECHANICAL ['A', 'GANG', 'MECHANICAL']
['BAL AIRE MECHANICAL INC', 'J S THOMAS MECHANICAL LLC', 'MECHANICAL SOLUTIONS CONSULTING LLC', 'P  M MECHANICAL HEATING  AIR CONDITIONING LLC']
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
=====


MULTIPLE
A GENERAL PAINTING  SERVICES INC ['A', 'PAINTING']
['ALEXS PAINTING INC', 'BROCKMAN PAINTING', 'COTA S PAINTING', 'DOUBLE J PLASTERING  PAINTING INC', 'ENDECCA PAINTING CO INC', 'GUILLON HONEYCUTT PAINTING LLC', 'SPILLIARD PAINTING LLC', 'STYLE PAINTING  TILE']
[0.29, 0.22, 0.29, 0.0, 0.4, 0.29, 0.0, 0.29]
[3.0, 3.0, 4.0, 2.0, 4.0, 0.0, 4.0, 1.0]
=====


MULTIPLE
A GEORGIA CUTTER INC ['A', 'GEORGIA', 'CUTTER']
['GEORGIA FREIGHT SYSTEM LLC', 'PALMS OF WEST GEORGIA LLC', 'PREMIER CONTRACTING SOLUTIONS OF GEORGIA']
[0.0, 0.22, 0.18]
[2.0, 3.0, 2.0]
=====


MULTIPLE
A GOOD SIGN  GRAPHICS CO ['A', 'GOOD', 'SIGN', 'GRAPHICS']
['SIGN POST STUDIO LLC', 'THE GOOD GUYS SECURITY LLC']
[0.0, 0.29]
[3.0, 2.0]
=====


DIRECT
A GRADE ABOVE OTHER


MULTIPLE
A J DOGGETT TRUCKING ['A', 'J', 'DOGGETT', 'TRUCKING']
['A HARRIS TRUCKING CO INC', 'A S L V TRUCKING LLC', 'ALFREDO MORAN TRUCKING INC', 'BIG HORN EXPRESS TRUCKING LLC', 'C  S TRUCKING LLC', 'DRESSEL TRUCKING', 'DUNSTON TRUCKING', 'FARMER  SONS TRUCKING LLC', 'FIONA TRUCKING LLC', 'FOSTER  SONS TRUCKING LLC', 'GREEN DIAMONDS TRUCKING LLC', 'HONGDA TRUCKING INC', 'INTEGRITY TRUCKING LLC', 'J D M TRUCKING', 'KOVACHI TRUCKING LLC', 'LAND DWELLERS TRUCKING LLC', 'MILLER EXCAVATING  TRUCKING LLC', 'NALDO TRUCKING', 'ORTÍZ ON THE GO TRUCKING', 'P3 TRUCKING SOLUTIONS LLC', 'PILLIARD TRUCKING LLC', 'RDM TRUCKING INC', 'SMITH TRUCKING LLC', 'SNH TRUCKING  HAULING', 'STARSTRUCK TRUCKING', 'SYNERGY TRUCKING INC', 'TEFLONN TOUGH TRUCKING CORP']
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[3.0, 4.0, 0.0, 3.0, 1.0, 1.0, 4.0, 4.0, 2.0, 0.0, 2.0, 1.0, 3.0, 3.0, 4.0, 4.0, 2.0, 3.0, 1.0, 3.0, 4.0, 3.0,


MULTIPLE
A L R NEVADA INC ['A', 'L', 'R', 'NEVADA']
['ALR NEVADA INC', 'DIGISTREAM NEVADA INC', 'NEVADA RECYCLING CORP', 'NWEC NEVADA INC', 'V N T NEVADA']
[0.05, 0.21, 0.17, 0.13, 0.0]
[1.0, 4.0, 2.0, 1.0, 2.0]
=====


MULTIPLE
A LADYS DAY OUT ['A', 'LADYS', 'DAY', 'OUT']
['INSIDE  OUT PROPERTY INSPECTORS INC', 'KERSHAW IN HOME DAY CARE', 'LITTLE LEARNERS EDUCATIONAL DAY CARE LLC']
[0.29, 0.29, 0.4]
[1.0, 2.0, 4.0]
=====


MULTIPLE
A M A EXTERMINATING ['A', 'M', 'A', 'EXTERMINATING']
['BARNES EXTERMINATING', 'BOB KLEPAC EXTERMINATING SERVICE', 'JERRYS EXTERMINATING CO', 'POWELL EXTERMINATING CO', 'PREFERRED GENERAL CONTRACTING AND EXTERMINATING', 'RCJ EXTERMINATING LLC']
[0.18, 0.46, 0.5, 0.22, 0.18, 0.18]
[3.0, 3.0, 2.0, 1.0, 3.0, 4.0]
=====


DIRECT
A M C TRANSFER INC ['A', 'M', 'C', 'TRANSFER']
['ABBONIZIO TRANSFER INC']
[0.46]
[2.0]
=====


MULTIPLE
A M COHRON & SON INC ['A', 'M', 'COHRON', 'SON']
['A M COHRON  SON INC', 'EVANS  SON INC', 'GENE PTACEK  SON FIRE EQUIPMENT CO INC',


MULTIPLE
A P C PEST CONTROL ['A', 'P', 'C', 'PEST']
['COMMON GROUND LAWN  PEST SOLUTIONS LLC', 'EMPYREAN PEST MANAGEMENT INC', 'HICKS PEST CONTROL', 'MCFADDEN LAWN  PEST MANAGEMENT LLC', 'RAMPART PEST SOLUTIONS LLC']
[0.0, 0.0, 0.0, 0.0, 0.0]
[3.0, 0.0, 1.0, 3.0, 1.0]
=====


MULTIPLE
A P FIRE PROTECTION LLC ['A', 'P', 'PROTECTION']
['247 FIRE PROTECTION SERVICES INCORPORATED', 'ABSOLUTE FIRE PROTECTION INC', 'ACCESS FIRE PROTECTION', 'AE FIRE PROTECTION', 'ALL IN ONE FIRE PROTECTION INC', 'AMADOR FIRE PROTECTION DISTRICT', 'ASPIS PROTECTION SERVICE LLC', 'BIG RED FIRE PROTECTION OF TEXAS LLC', 'BOULDER MOUNTAIN FIRE PROTECTION DISTRICT', 'C  T FIRE PROTECTION INC', 'CHESAPEAKE PROTECTION SERVICES INC', 'COLBY FIRE PROTECTION INC', 'DIAMOND FIRE PROTECTION CO IN', 'DONALD WESCOTT FIRE PROTECTION DISTRICT', 'GROSE FIRE PROTECTION INCORPORATED', 'HABITAT PROTECTION INC', 'JASPER THOMPSON LIGHTNING PROTECTION INC', 'LAS ANIMAS BENT COUNTY FIRE PROTECTION DISTRICT', 'MATCO FIRE PROTECTION


MULTIPLE
A PLUS TOWING  RECOVERY ['A', 'PLUS', 'TOWING']
['AKULA AUTOMOTIVE  TOWING INC', 'ALL PURPOSE TOWING LLC', 'ANTHONYS TOWING LIMITED LIABILITY COMPANY', 'AZTEC TOWING INC', 'B S R TOWING AND TRANSPORT', 'BRIANS TOLEDO TOWING SERVICE', 'COBEY TOWING SERVICE LLC', 'ELLIOTTS PLUS  MORE LLC', 'L R S TOWING', 'LOCKETT AUTO AND TOWING LLC', 'MARSHALL TOWING AND LOGISTICS LLC', 'MC TOWING AND RECOVERY LL', 'NUNN BETTER TOWING  RECOVERY LLC', 'REEDS TOWING (51)', 'ROBS TOWING  RECOVERY LLC', 'SPLIT SECOND TOWING  RECOVERY LLC', 'STEADFAST TOWING  RECOVERY LLC']
[0.0, 0.0, 0.4, 0.4, 0.4, 0.0, 0.0, 0.0, 0.4, 0.0, 0.4, 0.4, 0.4, 0.0, 0.0, 0.4, 0.4]
[3.0, 2.0, 2.0, 0.0, 1.0, 3.0, 3.0, 1.0, 1.0, 1.0, 3.0, 3.0, 2.0, 4.0, 4.0, 2.0, 4.0]
=====


MULTIPLE
A POWER ELECTRIC SERVICE CORPO ['A', 'POWER', 'CORPO']
['ADVANCED POWER ELECTRICAL CONTRACTORS INC', 'AR POWER AND CONTROLS LLC', 'C  C POWER INC', 'C R POWER', 'CONNECTED POWER SERVICES LLC', 'CONTRACTORS POWER  LIGHT COMPANY', 'DIVERSIFIED 

In [0]:
df_matches = pd.DataFrame(_direct_customer)
df_matches.columns = ['CUSTOMER']
df_matches["MATCH_CUSTOMER"] = _direct_match
df_matches["DRAW_UP_DATE"] = _direct_draw_up_date
df_matches.head()

In [100]:
df_multiple_matches = pd.DataFrame(_multiple_customer)
df_multiple_matches.columns = ['CUSTOMER']
df_multiple_matches["MATCH_CUSTOMER"] = _multiple_matches
df_multiple_matches["DRAW_UP_DATE"] = _multiple_drop_dates
df_multiple_matches.head()

,CUSTOMER,MATCH_CUSTOMER
0,1 AFFORDABLE LAWN MAINTENANCE,"[1 AFFORDABLE LAWN MAINTENANCE LLC, 5 STAR MAI..."
1,1 G R V LLC,"[B R CONSTRUCTION LLC, B AND R PLUMBING LLC, ..."
2,1 KC LOGISTICS,"[1ST ONE LOGISTICS CORPORATION, 24 7 LOGISTICS..."
3,1 PRIORITY ENVIRONMENTAL SERVICES LLC,"[CORE ENVIRONMENTAL SOLUTIONS, JAG ENVIRONMENT..."
4,1 WAY AUTO AND FREIGHT LOGISTICS,"[1845 LOGISTICS LLC, 230 LOGISTICS LLC, 7FOLDS..."
5,"1 WITH NATURE, LLC","[ANYTHING WITH INK WESTAZ LLC, FLEAUX WITH MO]"
6,10 33 AMBULANCE SERVICE LTD(04,"[1033 AMBULANCE SERVICE LTD, ALLEN COUNTY AMBU..."
7,1005 FINANCIAL GROUP,"[AGZ FINANCIAL SERVICES LLC, BRIGHTER BEGINNIN..."
8,1021 AUTO SALES LLC,"[AR SALES LLC, AUTO DETAILING SOLUTIONS OF ABI..."


In [0]:
MATCHES_1_TO_N_FOR_MANUAL_REVIEW_df = df_multiple_matches
MACTHES_1_TO_N_FOR_MANUAL_REVIEW = dataiku.Dataset("MATCHES_1_TO_N_FOR_MANUAL_REVIEW")
MACTHES_1_TO_N_FOR_MANUAL_REVIEW.write_with_schema(MATCHES_1_TO_N_FOR_MANUAL_REVIEW_df)

MATCHES_1_TO_1_df = df_matches
MATCHES_1_TO_1 = dataiku.Dataset("MATCHES_1_TO_1")
MATCHES_1_TO_1.write_with_schema(MATCHES_1_TO_1_df)